In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import glob
import torch
import numpy as np

from torchvision import transforms

/pfs/data5/home/tu/tu_tu/tu_zxoau27/penv/lib64/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
imgs = glob.glob('/pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/2_Ortho_RGB/*tif')
labels = glob.glob("/pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/Labels_all/*tif")

# create dictionary -> {key: 'link/to/image_or_label'}
labels_dict = {label.split("/")[-1].split(".")[0].rsplit('_', 1)[0] : label for label in labels}
imgs_dict = {img.split("/")[-1].split(".")[0].rsplit('_', 1)[0] : img for img in imgs}

# check if keys in both dicts are the same 
print(set(imgs_dict) == set(labels_dict))

keys = sorted(list(set(imgs_dict)))
print(keys[:5])

print(labels_dict[keys[0]])
print(imgs_dict[keys[0]])

True
['top_potsdam_2_10', 'top_potsdam_2_11', 'top_potsdam_2_12', 'top_potsdam_2_13', 'top_potsdam_2_14']
/pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/Labels_all/top_potsdam_2_10_label.tif
/pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/2_Ortho_RGB/top_potsdam_2_10_RGB.tif


In [24]:
def slice_images(keys, image_dict, label_dict, num_images):
    
    for idx, _ in enumerate(keys): 
        img2tensor = transforms.ToTensor()
        tensor2image = transforms.ToPILImage()
        
        image = Image.open(imgs_dict[keys[idx]])
        label = Image.open(labels_dict[keys[idx]])
        
        img_tensor = img2tensor(image)
        label_tensor = img2tensor(label)
        
        pxls = int(np.floor(6000 / num_images))
        
        img_patches = img_tensor.data.unfold(0, 3, 3).unfold(1, pxls, pxls).unfold(2, pxls, pxls).squeeze(0)
        label_patches = label_tensor.data.unfold(0, 3, 3).unfold(1, pxls, pxls).unfold(2, pxls, pxls).squeeze(0)
        
        for i in range(img_patches.shape[0]):
            for j in range(img_patches.shape[1]):
                sliced_image = tensor2image(img_patches[i][j])
                sliced_label = tensor2image(label_patches[i][j])
                
                img_dir = imgs_dict[keys[idx]].rsplit('/', 1)[0] + '/sliced/' + str(i) + str(j) + '_' + imgs_dict[keys[idx]].rsplit('/', 1)[1]
                label_dir = labels_dict[keys[idx]].rsplit('/', 1)[0] + '/sliced/' + str(i) + str(j) + '_' + labels_dict[keys[idx]].rsplit('/', 1)[1]
                
                sliced_image.save(img_dir)
                sliced_label.save(label_dir)
                
                print(f"Saved image to: {img_dir}")
                print(f"Saved label to: {label_dir}")

In [26]:
slice_images(keys, imgs_dict, labels_dict, 3)

Saved image to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/2_Ortho_RGB/sliced/00_top_potsdam_2_10_RGB.tif
Saved label to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/Labels_all/sliced/00_top_potsdam_2_10_label.tif
Saved image to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/2_Ortho_RGB/sliced/01_top_potsdam_2_10_RGB.tif
Saved label to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/Labels_all/sliced/01_top_potsdam_2_10_label.tif
Saved image to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/2_Ortho_RGB/sliced/02_top_potsdam_2_10_RGB.tif
Saved label to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/Labels_all/sliced/02_top_potsdam_2_10_label.tif
Saved image to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/model_training/2_Ortho_RGB/sliced/10_top_potsdam_2_10_RGB.tif
Saved label to: /pfs/work7/workspace/scratch/tu_zxobe27-ds_